In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df2 = pd.read_csv('../data/Raw/Raw_experiments_species.csv')
df2.shape

/tmp/ipykernel_75272/1873662471.py:1: DtypeWarning: Columns (23,26,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('../data/Raw/Raw_experiments_species.csv')


(12798, 124)

In [3]:
print("LISTA COMPLETA DE VARIABLES")
for i, columna in enumerate(df2.columns):
    print(f"{i+1:2d}. {columna}")

LISTA COMPLETA DE VARIABLES
 1. Source
 2. DOI
 3. Authors
 4. Year
 5. Technical "Red Criteria"
 6. Risk Assessment "Red Criteria"
 7. Species
 8. Organism Group
 9. Environment
10. Life Stage
11. In vitro/in vivo
12. Sex
13. Estimated Body Length (cm)
14. Estimated Maximum Ingestible Size (mm)
15. Experiment Type
16. Exposure Route
17. Particle Mix?
18. Negative Control
19. Reference Particle
20. Exposure Media
21. Solvent
22. Detergent
23. pH
24. Salinity (ppt)
25. Temperature (Avg)
26. Temperature (Min)
27. Temperature (Max)
28. Exposure Duration (days)
29. Recovery (Days)
30. Acute/Chronic
31. Number of Doses
32. Replicates
33. Sample Size
34. Dosing Frequency
35. Chemicals Added
36. Added Chemical Dose (mg/L, nominal)
37. Added Chemical Dose (mg/L, measured)
38. particles/mL water (master)
39. particles/mL water (master), reported or converted
40. μg/mL water (master)
41. μg/mL water (master), reported or converted
42. μm^3/mL water (master)
43. μm^2/mL water (master)
44. μm^2/ug

In [4]:
# Columnas duplicadas
duplicadas = df2.columns[df2.T.duplicated()]
duplicadas

Index([], dtype='object')

In [5]:
df2.duplicated().sum()

np.int64(533)

In [6]:
df2 = df2.drop_duplicates().reset_index(drop=True)
df2.shape

(12265, 124)

In [7]:
# ELIMINACIÓN DE COLUMNAS INÚTILES / INCOMPLETAS

cols_to_drop = [

    # Measured / Nominal Dose Alternative (100% NA)
    "Measured Dose Alternative Category",
    "Measured Dose Alternative Type",
    "Measured Dose Alternative Type Units",
    "Nominal Dose Alternative Category",
    "Nominal Dose Alternative Type",
    "Nominal Dose Alternative Type Units",

    # Scores metodológicos (30 columnas)
    "Test Medium Score",
    "Administration Route Score",
    "Test Species Score",
    "Sample Size Score",
    "Control Group Score",
    "Exposure Duration Score",
    "Particle Size Score",
    "Particle Shape Score",
    "Polymer Type Score",
    "Source of Microplastics Score",
    "Data Reporting Score",
    "Chemical Purity Score",
    "Laboratory Preparation Score",
    "Background Contamination Score",
    "Exposure Verification Score",
    "Exposure Homogeneity Score",
    "Exposure Assessment Score",
    "Replication Score",
    "Number of Treatments Score",
    "Endpoints Score",
    "Food Availability Score",
    "Effect Thresholds Score",
    "Dose Response Score",
    "Concentration Range Score",
    "Aging and Biofouling Score",
    "Microplastic Diversity Score",
    "Exposure Time Score",

    # Variables ambientales eliminadas igual que en humanos
    "pH",
    "Salinity (ppt)",
    "Temperature (Min)",
    "Temperature (Max)",
]


cols_to_drop_existing = [c for c in cols_to_drop if c in df2.columns]

print(f"Columnas a eliminar ({len(cols_to_drop_existing)} encontradas):")
for c in cols_to_drop_existing:
    print(" -", c)

# Aplicar eliminación
df2 = df2.drop(columns=cols_to_drop_existing)

print("\n✔ Eliminación completada.")
print("Nuevo shape:", df2.shape)


Columnas a eliminar (37 encontradas):
 - Measured Dose Alternative Category
 - Measured Dose Alternative Type
 - Measured Dose Alternative Type Units
 - Nominal Dose Alternative Category
 - Nominal Dose Alternative Type
 - Nominal Dose Alternative Type Units
 - Test Medium Score
 - Administration Route Score
 - Test Species Score
 - Sample Size Score
 - Control Group Score
 - Exposure Duration Score
 - Particle Size Score
 - Particle Shape Score
 - Polymer Type Score
 - Source of Microplastics Score
 - Data Reporting Score
 - Chemical Purity Score
 - Laboratory Preparation Score
 - Background Contamination Score
 - Exposure Verification Score
 - Exposure Homogeneity Score
 - Exposure Assessment Score
 - Replication Score
 - Number of Treatments Score
 - Endpoints Score
 - Food Availability Score
 - Effect Thresholds Score
 - Dose Response Score
 - Concentration Range Score
 - Aging and Biofouling Score
 - Microplastic Diversity Score
 - Exposure Time Score
 - pH
 - Salinity (ppt)
 - Te

In [8]:
#nulos
with pd.option_context('display.max_rows', None):
    display(df2.isna().mean().sort_values(ascending=False) * 100)

Recovery (Days)                                                     97.342030
Added Chemical Dose (mg/L, measured)                                96.877293
Functional Group                                                    93.689360
μm^2/ug/kg sediment dry weight (master)                             92.710966
μm^2/kg sediment dry weight (master)                                91.178149
μm^3/kg sediment dry weight (master)                                91.178149
particles/kg sediment dry weight (master), reported or converted    91.178149
particles/kg sediment dry weight (master)                           91.178149
mg/kg sediment dry weight (master), reported or converted           90.183449
mg/kg sediment dry weight (master)                                  90.183449
Charge                                                              87.346107
Zeta Potential Media                                                86.946596
Zeta Potential (mV)                                             

In [9]:
# ELIMINACIÓN DE COLUMNAS IRRELEVANTES CON >90% NA

cols_to_drop = [
    "Recovery (Days)",
    "Added Chemical Dose (mg/L, measured)",
    "Functional Group",   # en este dataset tiene 94% NA → se elimina aquí
    "μm^2/ug/kg sediment dry weight (master)",
    "μm^2/kg sediment dry weight (master)",
    "μm^3/kg sediment dry weight (master)",
    "particles/kg sediment dry weight (master), reported or converted",
    "particles/kg sediment dry weight (master)",
    "mg/kg sediment dry weight (master), reported or converted",
    "mg/kg sediment dry weight (master)"
]

# Filtrar solo las que realmente existen en df (seguridad)
cols_existing = [c for c in cols_to_drop if c in df2.columns]

df2.drop(columns=cols_existing, inplace=True)

print("Columnas eliminadas:")
for c in cols_existing:
    print("✔", c)

print(f"\nTotal eliminadas: {len(cols_existing)}")
print(f"Shape actual: {df2.shape}")


Columnas eliminadas:
✔ Recovery (Days)
✔ Added Chemical Dose (mg/L, measured)
✔ Functional Group
✔ μm^2/ug/kg sediment dry weight (master)
✔ μm^2/kg sediment dry weight (master)
✔ μm^3/kg sediment dry weight (master)
✔ particles/kg sediment dry weight (master), reported or converted
✔ particles/kg sediment dry weight (master)
✔ mg/kg sediment dry weight (master), reported or converted
✔ mg/kg sediment dry weight (master)

Total eliminadas: 10
Shape actual: (12265, 77)


In [10]:
#nulos
with pd.option_context('display.max_rows', None):
    display(df2.isna().mean().sort_values(ascending=False) * 100)

Charge                                                87.346107
Zeta Potential Media                                  86.946596
Zeta Potential (mV)                                   84.932735
Effect Metric                                         83.848349
AF NOEC                                               66.530779
Tissue Distribution                                   49.270281
Uptake Validation Method                              46.220954
Particle Width (μm)                                   37.431716
μm^2/ug/mL water (master)                             33.118630
μm^3/mL water (master)                                32.719119
μm^2/mL water (master)                                32.719119
particles/mL water (master), reported or converted    32.026091
particles/mL water (master)                           32.026091
μg/mL water (master)                                  25.894823
μg/mL water (master), reported or converted           25.894823
Estimated Body Length (cm)              

In [11]:
df2['μm^2/ug/mL water (master)'].unique()

array([1.06350460e+15, 8.49273459e+15, 4.65343983e+12, ...,
       3.05504946e+14, 3.05504946e+15, 3.05504946e+16], shape=(1169,))

In [12]:
df2["charge_missing"] = df2["Charge"].isna().astype(int)
df2["Charge"] = df2["Charge"].fillna("unknown")

print("✔ Charge imputado con 'unknown'")

✔ Charge imputado con 'unknown'


In [13]:
# Zeta Potential Media

col = "Zeta Potential Media"

# Pasar a texto, minúsculas, quitar espacios
df2[col] = df2[col].astype(str).str.strip().str.lower().replace("nan", np.nan)

# Diccionario de normalización
zeta_media_map = {
    "milliq water": "milliq_water",
    "milli q": "milliq_water",
    "ultra.pure.water": "ultrapure_water",
    "ultrapure water": "ultrapure_water",

    "natural.seawater": "natural_seawater",
    "natural filtered seawater": "natural_seawater",

    "filtered.seawater": "filtered_seawater",
    "filtered seawater": "filtered_seawater",

    "artificial.seawater": "artificial_seawater",
    "artificial seawater": "artificial_seawater",

    "instant.ocean": "instant_ocean",

    "filtered.autoclaved.natural.seawater": "filtered_autoclaved_natural_seawater",
    "mq water and mussle culture media": "mq_water_mussel_culture",

    "artificial.seawater.00003%methyleneblue": "artificial_seawater_methylene_blue"
}

# Aplicar normalización parcial
df2[col] = df2[col].replace(zeta_media_map)

# Crear flag de missing
df2["zeta_media_missing"] = df2[col].isna().astype(int)

# Reemplazar NaN por not_reported
df2[col] = df2[col].fillna("not_reported")

print("✔ Zeta Potential Media normalizada e imputada")
print("Valores únicos finales:", df2[col].unique())


✔ Zeta Potential Media normalizada e imputada
Valores únicos finales: ['not_reported' 'natural_seawater' 'artificial_seawater'
 'moderately.hard.water' 'filtered_seawater' 'hbss'
 'filtered_autoclaved_natural_seawater' 'sterile.natural.seawater'
 'trisacetatephosphatemedium' 'ultrapure_water'
 'artificial_seawater_methylene_blue' 'uv.treated.filtered.seawater'
 'instant_ocean' 'mq_water_mussel_culture' 'hemolymph serum (mussel)'
 'filtered artificial water' 'milliq_water' 'aap medium'
 'immune cell exposure media' "dulbecco's modified eagle's medium"
 'zebrafish water' 'bbm + chorella' 'bbm + subcapita' 'asw']


In [14]:
# IMPUTACIÓN: Zeta Potential (mV)

col = "Zeta Potential (mV)"
group = "Polymer"

# 1. Convertir a numérico
df2[col] = pd.to_numeric(df2[col], errors='coerce')

# 2. Normalizar Polymer
df2[group] = df2[group].astype(str).str.strip()

# 3. Flag de missing original
df2["zeta_missing"] = df2[col].isna().astype(int)

# 4. Mediana por polymer
median_by_poly = df2.groupby(group)[col].median()

# 5. Columna auxiliar: mediana correspondiente a cada fila
df2["zeta_median_by_poly"] = df2[group].map(median_by_poly)

# 6. Mediana global (fallback)
global_median = df2[col].median()

# 7. Imputación final
df2[col] = df2[col].fillna(df2["zeta_median_by_poly"])
df2[col] = df2[col].fillna(global_median)

# 8. Flags de trazabilidad
df2["zeta_imputed_by_group"] = (
    (df2["zeta_missing"] == 1) & df2["zeta_median_by_poly"].notna()
).astype(int)

df2["zeta_imputed_by_global"] = (
    (df2["zeta_missing"] == 1) & df2["zeta_median_by_poly"].isna()
).astype(int)

# 9. Limpiar columna auxiliar
df2 = df2.drop(columns=["zeta_median_by_poly"])

# 10. Resumen
print("=== Imputación Zeta Potential (mV) ===")
print("Filas totales:", len(df2))
print("Missing iniciales:", df2['zeta_missing'].sum())
print("Imputadas por mediana de polymer:", df2['zeta_imputed_by_group'].sum())
print("Imputadas por mediana global:", df2['zeta_imputed_by_global'].sum())
print("Missing restantes:", df2[col].isna().sum())
print("Mediana global usada como fallback:", global_median)


=== Imputación Zeta Potential (mV) ===
Filas totales: 12265
Missing iniciales: 10417
Imputadas por mediana de polymer: 8916
Imputadas por mediana global: 1501
Missing restantes: 0
Mediana global usada como fallback: -13.71


In [15]:
col = "Effect Metric"

# Normalizar a texto
df2[col] = df2[col].astype(str).str.strip().replace("nan", np.nan)

# Flag de missing original
df2["effect_metric_missing"] = df2[col].isna().astype(int)

# Imputación con not_reported
df2[col] = df2[col].fillna("not_reported")

print("✔ Effect Metric imputado con 'not_reported'")
print("Valores únicos:", df2[col].unique())


✔ Effect Metric imputado con 'not_reported'
Valores únicos: ['not_reported' 'NOEC' 'LOEC' 'LC50' 'HONEC' 'EC50' 'IC50' 'EC10' 'EC20'
 'LC20']


In [16]:
col = "AF NOEC"

# Normalizar la columna a texto
df2[col] = df2[col].astype(str).str.strip().replace("nan", np.nan)

# Flag de missing original
df2["af_noec_missing"] = df2[col].isna().astype(int)

# Imputación con 'not_reported'
df2[col] = df2[col].fillna("not_reported")

print("✔ AF NOEC imputado con 'not_reported'")
print("Valores únicos:", df2[col].unique())


✔ AF NOEC imputado con 'not_reported'
Valores únicos: ['2.0' 'not_reported' '1.0' '10.0']


In [17]:
# En los otros dos datasets se ha eliminado por: 
# No existe forma razonable de unificar más de 40 tejidos diferentes
# No es una variable primaria para predecir toxicidad
df2 = df2.drop(columns=["Tissue Distribution"])
print("✔ Tissue Distribution eliminada")


✔ Tissue Distribution eliminada


In [18]:
# Se elimina porque no existe en los otros datasets
df2 = df2.drop(columns=["Uptake Validation Method"])
print("✔ Uptake Validation Method eliminada")


✔ Uptake Validation Method eliminada


In [19]:
# Imputación Particle Width (μm)

col = "Particle Width (μm)"
group_col = "Shape"

# 1. Asegurar numérico
df2[col] = pd.to_numeric(df2[col], errors="coerce")

# 2. Normalizar Shape
df2[group_col] = df2[group_col].astype(str).str.strip().replace({"nan": np.nan})

# 3. Flag de missing original
df2["particle_width_missing"] = df2[col].isna().astype(int)

# 4. Mediana global
median_global = df2[col].median()

# 5. Mediana por shape
median_by_shape = df2.groupby(group_col)[col].median()

# 6. Crear función de imputación
def impute_width(row):
    if pd.notna(row[col]):
        return row[col]
    shape = row[group_col]
    if pd.notna(shape) and pd.notna(median_by_shape.get(shape, np.nan)):
        return median_by_shape[shape]
    return median_global

# 7. Aplicar imputación
df2[col] = df2.apply(impute_width, axis=1)

# 8. Confirmación
print("✔ Particle Width imputado correctamente")
print("Missing tras imputación:", df2[col].isna().sum())


✔ Particle Width imputado correctamente
Missing tras imputación: 0


In [20]:
cols_to_drop_master = [
    "μm^2/ug/mL water (master)",
    "μm^3/mL water (master)",
    "μm^2/mL water (master)"
]

# Filtrar solo las que realmente existen
cols_to_drop_master = [c for c in cols_to_drop_master if c in df2.columns]

df2 = df2.drop(columns=cols_to_drop_master)

print("Columnas eliminadas:", cols_to_drop_master)


Columnas eliminadas: ['μm^2/ug/mL water (master)', 'μm^3/mL water (master)', 'μm^2/mL water (master)']


In [21]:
# ELIMINAR TODAS LAS COLUMNAS "reported or converted"

cols_reported = [c for c in df2.columns if "reported or converted" in c.lower()]

df2 = df2.drop(columns=cols_reported)

print("Columnas eliminadas (‘reported or converted’):")
for c in cols_reported:
    print(" -", c)

print("\nTotal eliminadas:", len(cols_reported))


Columnas eliminadas (‘reported or converted’):
 - particles/mL water (master), reported or converted
 - μg/mL water (master), reported or converted

Total eliminadas: 2


In [22]:
col = "particles/mL water (master)"

# Convertir a numérico
df2[col] = pd.to_numeric(df2[col], errors="coerce")

# Crear flag de missing
df2["particles_ml_missing"] = df2[col].isna().astype(int)

print("✔ particles/mL water (master) tratado correctamente")
print("NAs:", df2[col].isna().sum())


✔ particles/mL water (master) tratado correctamente
NAs: 3928


In [23]:
col = "μg/mL water (master)"

# Convertir a numérico
df2[col] = pd.to_numeric(df2[col], errors="coerce")

# Crear flag de missing
df2["ug_ml_water_missing"] = df2[col].isna().astype(int)

print("✔ Flag creado para μg/mL water (master)")
print("NAs:", df2[col].isna().sum())
print("Valores únicos (sin NA):", sorted(df2[col].dropna().unique()[:10]), "...")


✔ Flag creado para μg/mL water (master)
NAs: 3176
Valores únicos (sin NA): [np.float64(0.00139), np.float64(0.005993938), np.float64(0.0111), np.float64(0.059939378), np.float64(0.599393782), np.float64(2.996968912), np.float64(5.993937824), np.float64(11.98787565), np.float64(27.57211399), np.float64(59.93937824)] ...


In [24]:
# Imputación Estimated Body Length (cm)

col = "Estimated Body Length (cm)"
group = "Species"

# Convertir a numérico correctamente
df2[col] = pd.to_numeric(df2[col], errors="coerce")

# Flag de missing original
df2["estimated_body_length_missing"] = df2[col].isna().astype(int)

# Calcular mediana por Species
median_by_species = df2.groupby(group)[col].median()

# Imputar por mediana de la especie
mask_missing = df2[col].isna()
df2.loc[mask_missing, col] = df2.loc[mask_missing, group].map(median_by_species)

# Fallback global por si quedan NA
global_median = df2[col].median(skipna=True)
df2[col] = df2[col].fillna(global_median)

# Resumen
print("=== Imputación Estimated Body Length (cm) ===")
print("Missing original:", df2['estimated_body_length_missing'].sum())
print("Mediana global usada como fallback:", global_median)
print("Missing tras imputación:", df2[col].isna().sum())
print("Valores únicos (primeros 20):", df2[col].unique()[:20])


=== Imputación Estimated Body Length (cm) ===
Missing original: 3138
Mediana global usada como fallback: 4.0
Missing tras imputación: 0
Valores únicos (primeros 20): [5.00e-01 8.00e-02 7.80e-01 1.50e+01 9.00e-02 4.00e+00 8.50e-01 1.20e+00
 1.00e+00 1.10e+01 2.50e+00 2.49e-02 3.00e+00 3.50e-01 1.40e+01 3.00e-01
 9.00e+00 9.00e-03 3.50e+01 9.80e+00]


In [25]:
# Imputación Estimated Maximum Ingestible Size (mm)

col = "Estimated Maximum Ingestible Size (mm)"
group = "Species"

# Convertir a numérico
df2[col] = pd.to_numeric(df2[col], errors="coerce")

# Flag de missing original
df2["estimated_max_ingestible_missing"] = df2[col].isna().astype(int)

# Mediana por Species
median_by_species = df2.groupby(group)[col].median()

# Imputación por especie
mask_missing = df2[col].isna()
df2.loc[mask_missing, col] = df2.loc[mask_missing, group].map(median_by_species)

# Fallback global
global_median = df2[col].median(skipna=True)
df2[col] = df2[col].fillna(global_median)

# Resumen
print("=== Imputación Estimated Maximum Ingestible Size (mm) ===")
print("Missing originales:", df2["estimated_max_ingestible_missing"].sum())
print("Mediana global usada en fallback:", global_median)
print("Missing tras imputación:", df2[col].isna().sum())
print("Valores únicos (primeros 20):", df2[col].unique()[:20])


=== Imputación Estimated Maximum Ingestible Size (mm) ===
Missing originales: 3138
Mediana global usada en fallback: 2.3794919391134903
Missing tras imputación: 0
Valores únicos (primeros 20): [3.41120548e-01 6.15852001e-02 5.16779846e-01 8.17874834e+00
 6.87476612e-02 2.37949194e+00 5.59977037e-01 7.72793040e-01
 6.51778454e-01 6.12159942e+00 1.53396610e+00 2.07009278e-02
 1.81877495e+00 2.44463463e-01 7.66828416e+00 2.11679579e-01
 5.07525571e+00 8.00125162e-03 1.80473716e+01 5.49546286e+00]


In [26]:
# Imputación Particle Mass (mg) — mediana por Polymer 

col2 = "Particle Mass (mg)"
group_col2 = "Polymer"

# Convertir a numérico
df2[col2] = pd.to_numeric(df2[col2], errors="coerce")

# Normalizar Polymer (importante!)
df2[group_col2] = df2[group_col2].astype(str).str.strip()

# Flag de missing original
df2["particle_mass_missing"] = df2[col2].isna().astype(int)

# Medianas por polymer
medians_by_poly = df2.groupby(group_col2)[col2].median()

# Crear columna auxiliar con la mediana asignada a cada fila
df2["mass_median_by_poly"] = df2[group_col2].map(medians_by_poly)

# Mediana global
global_median = df2[col2].median()

# Imputación final
df2[col2] = df2[col2].fillna(df2["mass_median_by_poly"])
df2[col2] = df2[col2].fillna(global_median)

# Flags de imputación
df2["particle_mass_imputed_by_group"] = (
    (df2["particle_mass_missing"] == 1) & df2["mass_median_by_poly"].notna()
).astype(int)

df2["particle_mass_imputed_by_global"] = (
    (df2["particle_mass_missing"] == 1) & df2["mass_median_by_poly"].isna()
).astype(int)

# Limpiar auxiliar
df2 = df2.drop(columns=["mass_median_by_poly"])

# Resumen
print("=== Imputación Particle Mass (mg) ===")
print("Total filas:", len(df2))
print("Missing originales:", df2["particle_mass_missing"].sum())
print("Imputadas por mediana Polymer:", df2["particle_mass_imputed_by_group"].sum())
print("Imputadas por mediana Global:", df2["particle_mass_imputed_by_global"].sum())
print("Missing tras imputación:", df2[col2].isna().sum())
print("Mediana global:", global_median)


=== Imputación Particle Mass (mg) ===
Total filas: 12265
Missing originales: 2386
Imputadas por mediana Polymer: 1804
Imputadas por mediana Global: 582
Missing tras imputación: 0
Mediana global: 2.225556648040447e-07


In [27]:
# Imputación Particle Volume (μm³) — mediana por Size Category

vol_col = "Particle Volume (μm^3)"
group_col = "Size Category"

# normalizar categoría
df2[group_col] = df2[group_col].astype(str).str.strip()
df2[group_col] = df2[group_col].replace({"nan": np.nan})

# convertir volumen a numérico
df2[vol_col] = pd.to_numeric(df2[vol_col], errors="coerce")

# flag missing original
df2["particle_volume_missing"] = df2[vol_col].isna().astype(int)

# medianas por Size Category
medians_by_size = df2.groupby(group_col)[vol_col].median()

# asignar mediana según categoría
df2["vol_median_by_size"] = df2[group_col].map(medians_by_size)

# fallback global
global_median = df2[vol_col].median()

# imputación final
df2[vol_col] = df2[vol_col].fillna(df2["vol_median_by_size"])
df2[vol_col] = df2[vol_col].fillna(global_median)

# limpiar auxiliar
df2 = df2.drop(columns=["vol_median_by_size"])

# resumen
print("=== Imputación Particle Volume (μm³) ===")
print("Missing originales:", df2["particle_volume_missing"].sum())
print("Missing tras imputación:", df2[vol_col].isna().sum())
print("Mediana global usada:", global_median)


=== Imputación Particle Volume (μm³) ===
Missing originales: 1864
Missing tras imputación: 0
Mediana global usada: 207.995948414995


In [28]:
# Imputación Particle Surface Area (μm²) — mediana por Size Category

area_col = "Particle Surface Area (μm^2)"
group_col = "Size Category"

# Normalizar categoría
df2[group_col] = df2[group_col].astype(str).str.strip()
df2[group_col] = df2[group_col].replace({"nan": np.nan})

# Convertir a numérico
df2[area_col] = pd.to_numeric(df2[area_col], errors="coerce")

# Flag missing original
df2["particle_area_missing"] = df2[area_col].isna().astype(int)

# Medianas por size category
medians_by_size = df2.groupby(group_col)[area_col].median()

# Map a cada fila
df2["area_median_by_size"] = df2[group_col].map(medians_by_size)

# Mediana global
global_median = df2[area_col].median()

# Imputación en cascada
df2[area_col] = df2[area_col].fillna(df2["area_median_by_size"])
df2[area_col] = df2[area_col].fillna(global_median)

# Borrar auxiliar
df2 = df2.drop(columns=["area_median_by_size"])

print("=== Imputación Particle Surface Area (μm²) ===")
print("Missing originales:", df2["particle_area_missing"].sum())
print("Missing tras imputación:", df2[area_col].isna().sum())
print("Mediana global usada:", global_median)


=== Imputación Particle Surface Area (μm²) ===
Missing originales: 1864
Missing tras imputación: 0
Mediana global usada: 189.43354210258192


In [29]:
col_cat = 'Density, reported or estimated'
col_num = 'Density (g/cm^3)'

# Normalizar a string y pasar a minúsculas
df2[col_cat] = df2[col_cat].astype(str).str.strip().str.lower().replace({'nan': np.nan})

# Normalizar variantes (por si aparecen en el futuro)
df2[col_cat] = df2[col_cat].replace({
    'est': 'estimated',
    'estim': 'estimated',
    'rep': 'reported',
    'rpt': 'reported'
})

# Inferir reported si hay dato numérico pero falta categoría
mask_infer_reported = df2[col_cat].isna() & df2[col_num].notna()
df2.loc[mask_infer_reported, col_cat] = 'reported'

# Rellenar el resto con unknown
df2[col_cat] = df2[col_cat].fillna('unknown')

# Crear flags
df2['density_reported_flag']  = (df2[col_cat] == 'reported').astype(int)
df2['density_estimated_flag'] = (df2[col_cat] == 'estimated').astype(int)
df2['density_unknown_flag']   = (df2[col_cat] == 'unknown').astype(int)

# Convertir a categoría
df2[col_cat] = df2[col_cat].astype('category')

# Comprobación
print(df2[col_cat].value_counts(dropna=False))
print(df2[['density_reported_flag',
          'density_estimated_flag',
          'density_unknown_flag']].sum())


Density, reported or estimated
estimated    8595
reported     3290
unknown       380
Name: count, dtype: int64
density_reported_flag     3290
density_estimated_flag    8595
density_unknown_flag       380
dtype: int64


In [30]:
import re

col = 'Sample Size'
group_col = 'Experiment Type'

# --- Copia de seguridad ---
if 'Sample Size_original' not in df2.columns:
    df2['Sample Size_original'] = df2[col]

# --- Función para extraer el primer número de un string ---
def extract_first_number(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    s = s.replace(",", "")  # quitar millares
    m = re.search(r'[-+]?\d*\.?\d+', s)
    if m:
        try:
            return float(m.group(0))
        except:
            return np.nan
    return np.nan

# --- Extraer número ---
df2['Sample Size_num'] = df2['Sample Size_original'].apply(extract_first_number)

# --- Flag de missing original ---
df2['sample_size_missing'] = df2['Sample Size_num'].isna().astype(int)

# --- Detectar outliers (>100) ---
df2['sample_size_outlier'] = (df2['Sample Size_num'] > 100).astype(int)

# Reemplazar outliers por NaN para imputación correcta
df2.loc[df2['Sample Size_num'] > 100, 'Sample Size_num'] = np.nan

# --- Imputación por mediana de Experiment Type ---
df2['Sample Size_num'] = df2.groupby(group_col)['Sample Size_num'] \
                            .transform(lambda x: x.fillna(x.median()))

# --- Fallback global ---
global_median = df2['Sample Size_num'].median()
df2['Sample Size_num'] = df2['Sample Size_num'].fillna(global_median)

# Convertir a entero si todos los valores son enteros
if df2['Sample Size_num'].dropna().apply(float.is_integer).all():
    df2['Sample Size_num'] = df2['Sample Size_num'].astype('Int64')

# Reemplazar columna original
df2['Sample Size'] = df2['Sample Size_num']

print("Mediana global usada como fallback:", global_median)
print("Missing finales:", df2['Sample Size'].isna().sum())
print("Outliers detectados (>100):", df2['sample_size_outlier'].sum())
print("Ejemplo de valores finales:", df2['Sample Size'].unique()[:20])


Mediana global usada como fallback: 5.0
Missing finales: 0
Outliers detectados (>100): 640
Ejemplo de valores finales: [ 1. 10.  5.  2.  3. 15.  8.  4.  9. 20.  6. 18. 17. 23. 24. 12. 50. 14.
 16. 68.]


In [31]:
df2 = df2.drop(columns=["Sample Size_original"])
print("✔ Eliminada columna auxiliar Sample Size_original")

✔ Eliminada columna auxiliar Sample Size_original


In [32]:
# Normalizar Shape (quitar espacios, convertir "Not Reported" a NA)
df2['Shape'] = df2['Shape'].astype(str).str.strip()
df2['Shape'] = df2['Shape'].replace({'Not Reported': np.nan})

# Flag de missing
df2['particle_length_missing'] = df2['Particle Length (μm)'].isna().astype(int)

# Cálculo de medianas
median_global = df2['Particle Length (μm)'].median()
median_by_shape = df2.groupby('Shape')['Particle Length (μm)'].median()

# Función de imputación
def impute_particle_length(row):
    if pd.notna(row['Particle Length (μm)']):
        return row['Particle Length (μm)']
    shape = row['Shape']
    if pd.notna(shape) and pd.notna(median_by_shape.get(shape, np.nan)):
        return median_by_shape[shape]
    return median_global

# Aplicación
df2['Particle Length (μm)'] = df2.apply(impute_particle_length, axis=1)

print("✔ Particle Length imputado correctamente")
print("Missing tras imputación:", df2['Particle Length (μm)'].isna().sum())


✔ Particle Length imputado correctamente
Missing tras imputación: 0


In [33]:
# Crear flag de missing
df2["shape_missing"] = df2["Shape"].isna().astype(int)

# Imputar como categoría 'unknown'
df2["Shape"] = df2["Shape"].fillna("unknown")

# Convertir a categoría
df2["Shape"] = df2["Shape"].astype("category")

print("✔ Shape imputado como 'unknown'")
print(df2["Shape"].unique()[:20])


✔ Shape imputado como 'unknown'
['Sphere', 'Fragment', 'Fiber', 'unknown']
Categories (4, object): ['Fiber', 'Fragment', 'Sphere', 'unknown']


In [34]:
# Density (g/cm^3) — imputación por mediana global

col = "Density (g/cm^3)"

# Forzar numérico por seguridad
df2[col] = pd.to_numeric(df2[col], errors="coerce")

# Crear flag de missing
df2["density_missing"] = df2[col].isna().astype(int)

# Mediana global
density_median = df2[col].median()

# Imputación
df2[col] = df2[col].fillna(density_median)

print("✔ Density (g/cm^3) imputado correctamente")
print("Mediana global usada:", density_median)
print("Missing tras imputación:", df2[col].isna().sum())


✔ Density (g/cm^3) imputado correctamente
Mediana global usada: 1.07
Missing tras imputación: 0


In [35]:
col = "Dosing Frequency"

# Forzar a numérico
df2[col] = pd.to_numeric(df2[col], errors="coerce")

# Crear flag missing
df2["dosing_frequency_missing"] = df2[col].isna().astype(int)

# Calcular mediana global
median_dosing = df2[col].median()

# Imputar
df2[col] = df2[col].fillna(median_dosing)

print("✔ Dosing Frequency imputado correctamente")
print("Mediana usada:", median_dosing)
print("Missing tras imputación:", df2[col].isna().sum())


✔ Dosing Frequency imputado correctamente
Mediana usada: 1.0
Missing tras imputación: 0


In [36]:
col = "Size Validated?"

# Normalizar strings
df2[col] = df2[col].astype(str).str.strip().str.lower()

# Convertir variaciones a sí/no
df2[col] = df2[col].replace({
    "y": "yes",
    "yes": "yes",
    "n": "no",
    "no": "no",
    "nan": np.nan
})

# Crear flag de missing original
df2["size_validated_missing"] = df2[col].isna().astype(int)

# Imputar NA como 'unknown'
df2[col] = df2[col].fillna("unknown")

# Crear flags dummy
df2["size_validated_yes"] = (df2[col] == "yes").astype(int)
df2["size_validated_no"] = (df2[col] == "no").astype(int)
df2["size_validated_unknown"] = (df2[col] == "unknown").astype(int)

print("✔ Size Validated? imputado y normalizado correctamente")
print(df2[col].value_counts(dropna=False))


✔ Size Validated? imputado y normalizado correctamente
Size Validated?
yes        6523
no         5512
unknown     230
Name: count, dtype: int64


In [37]:
col = "Shape Validated"

# Normalizar strings
df2[col] = df2[col].astype(str).str.strip().str.lower()

# Convertir variaciones a sí/no
df2[col] = df2[col].replace({
    "y": "yes",
    "yes": "yes",
    "n": "no",
    "no": "no",
    "nan": np.nan
})

# Flag original de missing
df2["shape_validated_missing"] = df2[col].isna().astype(int)

# Imputación → unknown
df2[col] = df2[col].fillna("unknown")

# Crear dummies
df2["shape_validated_yes"] = (df2[col] == "yes").astype(int)
df2["shape_validated_no"] = (df2[col] == "no").astype(int)
df2["shape_validated_unknown"] = (df2[col] == "unknown").astype(int)

print("✔ Shape Validated imputado y normalizado correctamente")
print(df2[col].value_counts(dropna=False))


✔ Shape Validated imputado y normalizado correctamente
Shape Validated
yes        6511
no         5524
unknown     230
Name: count, dtype: int64


In [38]:
col = "Effect"

# normalizar strings
df2[col] = df2[col].astype(str).str.strip().str.lower()
df2[col] = df2[col].replace({"nan": np.nan})

# flag original de missing
df2["effect_missing"] = df2[col].isna().astype(int)

# imputación
df2[col] = df2[col].fillna("unknown")

# dummies opcionales
df2["effect_yes"] = (df2[col] == "yes").astype(int)
df2["effect_no"] = (df2[col] == "no").astype(int)
df2["effect_unknown"] = (df2[col] == "unknown").astype(int)

print("✔ Effect imputado correctamente")
print(df2[col].value_counts(dropna=False))


✔ Effect imputado correctamente
Effect
no         8011
yes        4104
unknown     150
Name: count, dtype: int64


In [39]:
col = "Uptake Validated?"

# Normalizar strings
df2[col] = df2[col].astype(str).str.strip().str.lower()
df2[col] = df2[col].replace({
    "yes": "yes",
    "y": "yes",
    "no": "no",
    "n": "no",
    "nan": np.nan
})

# Flag original de missing
df2["uptake_validated_missing"] = df2[col].isna().astype(int)

# Imputación
df2[col] = df2[col].fillna("unknown")

# Dummies opcionales
df2["uptake_yes"] = (df2[col] == "yes").astype(int)
df2["uptake_no"] = (df2[col] == "no").astype(int)
df2["uptake_unknown"] = (df2[col] == "unknown").astype(int)

print("✔ Uptake Validated? imputado correctamente")
print(df2[col].value_counts(dropna=False))


✔ Uptake Validated? imputado correctamente
Uptake Validated?
yes        6755
no         5415
unknown      95
Name: count, dtype: int64


In [40]:
col = "Species"

# Normalizar
df2[col] = df2[col].astype(str).str.strip()
df2[col] = df2[col].replace({"nan": np.nan, "NA NA": np.nan})

# Flag original de nulos
df2["species_missing"] = df2[col].isna().astype(int)

# Imputación simple
df2[col] = df2[col].fillna("unknown_species")

print("✔ Species imputado correctamente")
print(df2[col].value_counts().head())
print("NAs tras imputación:", df2[col].isna().sum())


✔ Species imputado correctamente
Species
Danio rerio                  1990
Oryzias melastigma           1022
Daphnia magna                 580
Oncorhynchus mykiss           560
Mytilus galloprovincialis     463
Name: count, dtype: int64
NAs tras imputación: 0


In [41]:
col = "Concentration Validated?"

# Normalizar
df2[col] = df2[col].astype(str).str.strip().str.lower()
df2[col] = df2[col].replace({
    "y": "yes", "n": "no",
    "yes": "yes", "no": "no",
    "nan": np.nan
})

# Flag original de NA
df2["concentration_validated_missing"] = df2[col].isna().astype(int)

# Imputación con unknown
df2[col] = df2[col].fillna("unknown")

print("✔ Concentration Validated? imputado correctamente")
print(df2[col].value_counts(dropna=False))
print("NAs tras imputación:", df2[col].isna().sum())


✔ Concentration Validated? imputado correctamente
Concentration Validated?
no         8917
yes        3304
unknown      44
Name: count, dtype: int64
NAs tras imputación: 0


In [42]:
col = "Exposure Duration (days)"

# Convertir a numérico
df2[col] = pd.to_numeric(df2[col], errors="coerce")

# Flag original de missing
df2["exposure_duration_missing"] = df2[col].isna().astype(int)

# Mediana global
median_global = df2[col].median()

# Imputación
df2[col] = df2[col].fillna(median_global)

print("✔ Exposure Duration imputado correctamente")
print("Mediana global usada:", median_global)
print("NAs tras imputación:", df2[col].isna().sum())


✔ Exposure Duration imputado correctamente
Mediana global usada: 14.0
NAs tras imputación: 0


In [43]:
col = "Size Category"

# Normalizar texto
df2[col] = df2[col].astype(str).str.strip()

# Convertir "Not Reported" → NaN
df2[col] = df2[col].replace({"Not Reported": np.nan, "nan": np.nan})

# Flag original de missing
df2["size_category_missing"] = df2[col].isna().astype(int)

# Imputación con categoría explícita
df2[col] = df2[col].fillna("unknown")

# Convertir a categoría
df2[col] = df2[col].astype("category")

print("✔ Size Category imputado con 'unknown'")
print("Valores finales:", df2[col].unique())


✔ Size Category imputado con 'unknown'
Valores finales: ['1µm < 100µm', '100µm < 1mm', '1nm < 100nm', '100nm < 1µm', '1mm < 5mm', 'unknown']
Categories (6, object): ['100nm < 1µm', '100µm < 1mm', '1mm < 5mm', '1nm < 100nm', '1µm < 100µm', 'unknown']


In [44]:
col = "Replicates"

# Convertir a string y limpiar
df2[col] = df2[col].astype(str).str.strip().replace({"nan": np.nan})

# Función para extraer primer número 
def extract_first_number(x):
    if pd.isna(x):
        return np.nan
    s = str(x)
    m = re.search(r"[-+]?\d*\.?\d+", s)
    if m:
        try:
            return float(m.group(0))
        except:
            return np.nan
    return np.nan

# Extraer número limpio
df2[col] = df2[col].apply(extract_first_number)

# Flag opcional para trazabilidad
df2["replicates_missing"] = df2[col].isna().astype(int)

# Imputación: moda global
mode_value = df2[col].mode()[0]
df2[col] = df2[col].fillna(mode_value)

# Convertir a entero si todos los valores lo permiten
if df2[col].dropna().apply(float.is_integer).all():
    df2[col] = df2[col].astype("Int64")

print("✔ Replicates imputado correctamente")
print("Moda global usada:", mode_value)
print("Valores finales únicos:", df2[col].unique())


✔ Replicates imputado correctamente
Moda global usada: 3.0
Valores finales únicos: <IntegerArray>
[ 8,  3,  6,  4, 12, 15,  5,  1,  2, 20, 10,  7,  9, 13, 24, 11, 45, 16, 14,
 17, 36, 33, 40, 30, 18, 27,  0]
Length: 27, dtype: Int64


In [45]:
#nulos
with pd.option_context('display.max_rows', None):
    display(df2.isna().mean().sort_values(ascending=False) * 100)

particles/mL water (master)               32.026091
μg/mL water (master)                      25.894823
Authors                                    0.000000
Year                                       0.000000
Source                                     0.000000
DOI                                        0.000000
Species                                    0.000000
Organism Group                             0.000000
Environment                                0.000000
Life Stage                                 0.000000
In vitro/in vivo                           0.000000
Sex                                        0.000000
Estimated Body Length (cm)                 0.000000
Estimated Maximum Ingestible Size (mm)     0.000000
Experiment Type                            0.000000
Exposure Route                             0.000000
Particle Mix?                              0.000000
Negative Control                           0.000000
Reference Particle                         0.000000
Exposure Med

In [46]:
df2['Replicates'].unique()

<IntegerArray>
[ 8,  3,  6,  4, 12, 15,  5,  1,  2, 20, 10,  7,  9, 13, 24, 11, 45, 16, 14,
 17, 36, 33, 40, 30, 18, 27,  0]
Length: 27, dtype: Int64

In [47]:
df2_processed = df2
df2_processed.to_csv("../data/processed/species_raw_processed.csv", index=False)

In [48]:
df2_processed.shape

(12265, 118)